In [23]:
!pip install tqdm numpy pandas matplotlib openai datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 12.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 19.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 39.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 48.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 37.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 33.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 32.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━

# Load Features from pickle

In [4]:
import pickle

with open('VGSG_train_features.pkl', 'rb') as f:
    train_data = pickle.load(f)

with open('VGSG_test_features.pkl', 'rb') as f:
    test_data = pickle.load(f)

with open('VGSG_val_features.pkl', 'rb') as f:
    val_data = pickle.load(f)

print(len(train_data), len(test_data), len(val_data))

/tmp/ipykernel_5814/1870838960.py:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  train_data = pickle.load(f)
/tmp/ipykernel_5814/1870838960.py:7: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case,

11773 586 849


In [48]:
def load_features(data):
    img_li=[]; sce_li=[]; swin_base_li=[]
    img_id_dic={}; sce_id_dic={}
    img_swin_base_dic={}; img_target_dic={}; img_origin_dic={}
    sce_img_target_dic={}; sce_img_origin_dic={}
    
    for id, story in data.items():
        scene_id=story[0]['scene_full_id']
        sce_li.append(scene_id)
        
        sce_img_target_dic[scene_id]=dict()
        sce_img_origin_dic[scene_id]=dict()

        for i, image in enumerate(story):
            image_id=image['image_id']

            if image_id not in img_swin_base_dic:
                img_li.append(image_id)
                swin_base_li.append(image['swin_base'])
                img_swin_base_dic[image_id]=image['swin_base']
                
            img_target_dic.setdefault(image_id, []).append(image['target'])
            img_origin_dic.setdefault(image_id, []).append(image['original_text'])

            sce_img_target_dic[scene_id][image_id]=image['target']
            sce_img_origin_dic[scene_id][image_id]=image['original_text']

    img_id_dic={image_id:i for i, image_id in enumerate(img_li)}
    sce_id_dic={scene_id:id for id, scene_id in enumerate(sce_li)}

    return img_li, sce_li, swin_base_li, img_id_dic, sce_id_dic, \
    img_swin_base_dic, img_target_dic, img_origin_dic, sce_img_target_dic, sce_img_origin_dic

In [19]:
train_img_li, train_sce_li, train_swin_base_li, train_img_id_dic, train_sce_id_dic, train_img_swin_base_dic, \
train_img_target_dic, train_img_origin_dic, train_sce_img_target_dic, train_sce_img_origin_dic=load_features(train_data)

test_img_li, test_sce_li, test_swin_base_li, test_img_id_dic, test_sce_id_dic, test_img_swin_base_dic, \
test_img_target_dic, test_img_origin_dic, test_sce_img_target_dic, test_sce_img_origin_dic=load_features(test_data)

val_img_li, val_sce_li, val_swin_base_li, val_img_id_dic, val_sce_id_dic, val_img_swin_base_dic, \
val_img_target_dic, val_img_origin_dic, val_sce_img_target_dic, val_sce_img_origin_dic=load_features(val_data)

In [52]:
import pickle

with open('outputs/train_features.json', 'wb') as fd:
    pickle.dump([train_img_li, train_sce_li, train_swin_base_li, \
                         train_img_id_dic, train_sce_id_dic, train_img_swin_base_dic, \
                         train_img_target_dic, train_img_origin_dic, \
                         train_sce_img_target_dic, train_sce_img_origin_dic],\
               fd, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('outputs/test_features.json', 'wb') as fd:
    pickle.dump([test_img_li, test_sce_li, test_swin_base_li, \
                         test_img_id_dic, test_sce_id_dic, test_img_swin_base_dic, \
                         test_img_target_dic, test_img_origin_dic, \
                         test_sce_img_target_dic, test_sce_img_origin_dic],\
               fd, protocol=pickle.HIGHEST_PROTOCOL)

with open('outputs/val_features.json', 'wb') as fd:
    pickle.dump([val_img_li, val_sce_li, val_swin_base_li, \
                         val_img_id_dic, val_sce_id_dic, val_img_swin_base_dic, \
                         val_img_target_dic, val_img_origin_dic, \
                         val_sce_img_target_dic, val_sce_img_origin_dic],\
               fd, protocol=pickle.HIGHEST_PROTOCOL)


In [46]:
print(len(train_img_li), len(train_sce_li), len(train_swin_base_li))
print(len(test_img_li), len(test_sce_li), len(test_swin_base_li))
print(len(val_img_li), len(val_sce_li), len(val_swin_base_li))

16494 11773 16494
1604 586 1604
1759 849 1759


# Compute the Similiarity

In [30]:
import torch
cos = torch.nn.CosineSimilarity(dim=1)
sim=cos(torch.Tensor(train_swin_base_li[0]), torch.Tensor(train_swin_base_li[1]))
print(sim, type(sim), sim.item(), type(sim.item()))

print(cos(torch.Tensor(train_swin_base_li[0]), torch.Tensor(train_swin_base_li[0])))
print(cos(torch.Tensor(train_swin_base_li[0]), torch.Tensor(train_swin_base_li[1])))

tensor([0.6747]) <class 'torch.Tensor'> 0.6746718287467957 <class 'float'>
tensor([1.0000])
tensor([0.6747])


In [35]:
import torch

def similarity_matrix(vec_li_1, vec_li_2):
    len_vec_1=len(vec_li_1)
    len_vec_2=len(vec_li_2)

    sim_matrix=[[0]*len_vec_2 for _ in range(len_vec_1)]
    cos = torch.nn.CosineSimilarity(dim=1)

    for idx, vec_1 in enumerate(vec_li_1):
        for i, vec_2 in enumerate(vec_li_2):
            sim=cos(torch.Tensor(vec_1), torch.Tensor(vec_2)).item()
            sim_matrix[idx][i]=sim
            
    return sim_matrix

In [36]:
val_train_sim=similarity_matrix(val_swin_base_li, train_swin_base_li)
test_train_sim=similarity_matrix(test_swin_base_li, train_swin_base_li)

In [45]:
print(len(val_train_sim), len(val_train_sim[0]))
print(len(test_train_sim), len(test_train_sim[0]))

1759 16494
1604 16494


In [53]:
import json

with open('val_train_sim.json', 'w') as fd:
    fd.write(json.dumps(val_train_sim)) 

with open('test_train_sim.json', 'w') as fd:
    fd.write(json.dumps(test_train_sim)) 

In [54]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

/root/miniconda3/envs/vgsg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniconda3/envs/vgsg/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [55]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):